# Twitter API

## Lecture Notes

#####Why Twitter ?


"While physics and math may tell us how the universe began, they are not much use in predicting Human Behavior because there are far too many Equations to Solve" 
    
    -Stephan Hawking
    

#### Rest API vs Streaming API: 


REST:  
    - Query user accounts using OAuth
    - Allows you to access 'historical' tweets

STREAM: 
    - Essentially long-running request (left Open) using OAuth
    - Access realtime stream of data
       

#### Rest API

In [ ]:
from __future__ import print_function

In [ ]:
# pip install requests_oauthlib
import requests
from requests_oauthlib import OAuth1

#OAuth ~ simple way to to publish & interact with data

In [ ]:
# Importing our Config
#pip install cnfg   ##  Create a config file in your home directory 

import cnfg

In [ ]:
from pprint import pprint

In [ ]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [ ]:
# ?? this is an instance 

response = requests.get("https://api.twitter.com/1.1/statuses/user_timeline.json",
                        auth=oauth)
tweets = response.json()

for key in tweets[0].keys():
    print(key)
    

In [ ]:
for tweet in tweets:
    print(tweet['text'], end="\n\n")

In [ ]:
parameters = {"q": ["rio", "olympics"], "count":40}
response = requests.get("https://api.twitter.com/1.1/search/tweets.json",
                        params = parameters,
                        auth=oauth)

pprint(response.json()['search_metadata'])

In [ ]:
tweets = response.json()['statuses']

print('PAGE 1')
for tweet in tweets:
    print(tweet['id'], tweet['text'], end="\n\n")

In [ ]:
response.json()['search_metadata']

In [ ]:
search_url = "https://api.twitter.com/1.1/search/tweets.json"
response_metadata = response.json()['search_metadata']

if "next_results" in response_metadata:
    next_page_url = search_url + response_metadata['next_results']

    response = requests.get(next_page_url, auth=oauth)

    print('PAGE 2')
    for tweet in response.json()['statuses']:
        print(tweet['text'], end="\n\n")

#### STREAMING API ~ TWEEPY

In [ ]:
# pip install tweepy
import tweepy
import pandas as pd

auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [ ]:

max_tweets=1

#Tweepy Cursor handles pagination .. 

for tweet in tweepy.Cursor(api.search,q=["rio", "olympics"]).items(max_tweets):
    pprint(tweet)

In [ ]:
results=[]


for tweet in tweepy.Cursor(api.search,q="olympics").items(10):
    results.append(tweet)

#### Import tweets into Pandas

In [ ]:
#  import pandas as pd
def structure_results(results):
    id_list=[tweet.id for tweet in results]
    data=pd.DataFrame(id_list,columns=['id'])
    
#     data["text"]= [tweet.text.encode('utf-8') for tweet in results]  # Python 2
    data["text"]= [tweet.text for tweet in results]  # Python 3
    data["datetime"]=[tweet.created_at for tweet in results]
    data["Location"]=[tweet.place for tweet in results]
    
    return data

In [ ]:
data=structure_results(results)
data.head()

#### Import Tweets into MongoDB

In [ ]:
import json
from pymongo import MongoClient


client = MongoClient()
db = client.legislation
# Our collection: 
tweets = db.new_tweet

In [ ]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert_one(data)

In [ ]:
tweets.find_one()